In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

In [3]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [4]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [15]:
#  metric使用

metric = keras.metrics.MeanSquaredError()
print(metric([5.], [2.]))
print(metric([4.], [2.]))  #  metric有累加效果
print(metric.result())

metric.reset_states()  #  清除累加
print(metric([4.], [2.]))
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.5, shape=(), dtype=float32)
tf.Tensor(6.5, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [17]:
#  1.batch 遍历数据集 metric
#     1.1 自动求导
#  2.epoch结束 验证数据集 metric

epochs = 100
batch_size = 32
steps_per_epoch = len(x_train_scaled) // batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()

def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size= batch_size)
    return x[idx], y[idx]

model = keras.Sequential([
    keras.layers.Dense(32, activation=tf.nn.relu,input_shape=x_train.shape[1:]),
    keras.layers.Dense(1)
])

for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch = random_batch(x_train_scaled, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = tf.reduce_mean(keras.losses.mean_squared_error(y_batch, y_pred))
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model.variables)
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch:", epoch, "train mse:", metric.result().numpy(), end=" ")
    y_vaild_pred = model(x_valid_scaled)
    vaild_loss = tf.reduce_mean(keras.losses.mean_squared_error(y_valid, y_vaild_pred))
    print("\t", "vaild mse:", vaild_loss.numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 0 train mse: 1.4155911 	 vaild mse: 1.46943337  0 train mse: 1.6705226  0 train mse: 1.5592906  0 train mse: 1.4709429  0 train mse: 1.4649867  0 train mse: 1.4702524  0 train mse: 1.4696999  0 train mse: 1.4515474 1.4328837  0 train mse: 1.4301254  0 train mse: 1.4189728 
Epoch: 1 train mse: 1.5986402 	 vaild mse: 1.43350777  1 train mse: 2.1390018  1 train mse: 1.9547393  1 train mse: 1.7923397  1 train mse: 1.7658286  1 train mse: 1.8823577  1 train mse: 2.4312477  1 train mse: 2.401092  1 tra

Epoch: 18 train mse: 1.2518859 	 vaild mse: 1.38615576524  18 train mse: 1.1607409  18 train mse: 1.1724759  18 train mse: 1.1761842  18 train mse: 1.2166624  18 train mse: 1.2128958  18 train mse: 1.2225771  18 train mse: 1.237911 train mse: 1.2418007  18 train mse: 1.2438759  18 train mse: 1.2461904  18 train mse: 1.239605  18 train mse: 1.2401295  18 train mse: 1.2464494  18 train mse: 1.2452071  18 train mse: 1.2493552 
Epoch: 19 train mse: 1.2550474 	 vaild mse: 1.385635408  19 train mse: 1.2749732  19 train mse: 1.2505207  19 train mse: 1.2690206  19 train mse: 1.2694433 19 train mse: 1.2776837  train mse: 1.2728988  19 train mse: 1.2650878 1.2454035  train mse: 1.2561934  19 train mse: 1.255018  19 train mse: 1.2574301 
Epoch: 20 train mse: 1.2366562  20 train mse: 1.240016  20 train mse: 1.2388598  20 train mse: 1.1829149  20 train mse: 1.1954257  1.2142439  20 train mse: 1.2242236  20 train mse: 1.2292529  20 train mse: 1.2295483  20 train mse: 1.2284087  20 train mse: 1.22850

KeyboardInterrupt: 